In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

In [2]:
# File to Load (Remember to Change These)
data_file = os.path.join("Resources", "purchase_data.csv")

In [3]:
# Read Purchasing File and store into Pandas data frame
data_file_df = pd.read_csv(data_file, encoding="utf-8")

In [4]:
data_file_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#Verify that there aren't missing entries
data_file_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [6]:
#Find the total number of unique players, create a data frame for the information, print out data frame
total_players = len(data_file_df["SN"].unique())
total_players_df = pd.DataFrame({"Total Players": [total_players]})
total_players_df

,Total Players
0,576


In [7]:
#For formatting money
def money (x):
    return "${:.2f}".format(x)

In [8]:
#For formatting percentages
def percent (x):
    return x.map('{:,.2%}'.format)

In [9]:
#Calculate the values needed for the "Purchasing Analysis (Total)" data frame
unique_items = len((data_file_df["Item Name"].unique()))
ave_price = data_file_df["Price"].mean()
num_purchases = len(data_file_df)
total_revenue = data_file_df["Price"].sum()

In [10]:
#Create and print out the purchase summary as a data frame
purchase_summ_df = pd.DataFrame({"Unique Items": [unique_items],
                                 "Average Price": "${:,.2f}".format(ave_price),
                                 "Number of Purchases": num_purchases,
                                 "Total Revenue": "${:,.2f}".format(total_revenue)})

purchase_summ_df

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [11]:
#Create dataframe for unique player info (SN, Age, Gender)
player_info_df = data_file_df.drop_duplicates(subset = "SN")
player_info_df = player_info_df.drop(["Purchase ID", "Item ID", "Item Name", "Price"], axis=1)

In [12]:
#Create dataframe for the analysis by gender, using unique player info only
gender_total = player_info_df["Gender"].value_counts()  #total number of unique players by gender (M + F + O)
gender_percent = (gender_total / total_players).map('{:,.2%}'.format)

gender_df = pd.DataFrame({"Total Count": gender_total,
                          "Percentage of Players": gender_percent})

gender_df.index.name = "Gender"
gender_df.sort_values(by = ["Gender"], ascending = True)
gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [13]:
# DF with pertinent gender and purchase information
gender_data_df = data_file_df[["Gender", "Price"]]

#Calculate Values for Purchase Count, using all purchase info
purchase_gender_count = gender_data_df["Gender"].value_counts()

#Create index and first column of gender analysis data frame, initialize dataframe here to cut down on variable names
gender_summ_df = pd.DataFrame({"Purchase Count": purchase_gender_count})
gender_summ_df.index.name = "Gender"

# Create the column for "Average Purchase Price"
gender_summ_df["Average Purchase Price"] = gender_data_df.groupby(["Gender"]).mean()

#Create the column for "Total Purchase Value"
gender_summ_df["Total Purchase Value"] = gender_data_df.groupby(["Gender"]).sum()

#Create the column for Avg Purchase Total per Person by Gender (Total Purchase Value / Total Players)
gender_summ_df["Avg Total Purchase per Person"] = gender_summ_df["Total Purchase Value"] / gender_total

#Format Data Frame
gender_summ_df["Average Purchase Price"] = gender_summ_df["Average Purchase Price"].map("${:,.2f}".format)
gender_summ_df["Total Purchase Value"] = gender_summ_df["Total Purchase Value"].map("${:,.2f}".format)
gender_summ_df["Avg Total Purchase per Person"] = gender_summ_df["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Display Purchasing Analysis (Gender)
gender_summ_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [14]:
#Create age bins and labels
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
age_display = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_bins_unique = (pd.cut(player_info_df["Age"], age_bins, labels = age_display, include_lowest = True))

#Calculate the total players in each group, and the percent of the total (using unique player info)
age_total = age_bins_unique.value_counts()
age_percent = (age_total/total_players).map('{:,.2%}'.format)

#Create dataframe and sort it by the age ranges
age_df = pd.DataFrame({"Total Count": age_total,
                          "Percentage of Players": age_percent})

age_df.index.name = "Age Ranges"
age_df = age_df.sort_values(by = ["Age Ranges"])

#Display the Age Demographics data frame
age_df


,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [15]:
# DF with pertinent age and purchase information
age_data_df = data_file_df[["Age", "Price"]]

#Binned data, using all info
age_bins_all = pd.cut(age_data_df["Age"], age_bins, labels = age_display, include_lowest = True)
age_grouped = age_data_df.groupby(age_bins_all)["Age"]

#Calculate Values for Purchase Count
purchase_age_count = age_bins_all.value_counts()

#Create index and first column of age analysis data frame, initialize dataframe here to cut down on variable names
age_summ_df = pd.DataFrame({"Purchase Count": purchase_age_count})

# Create the column for "Average Purchase Price"
age_summ_df["Average Purchase Price"] = age_data_df.groupby(age_bins_all)["Price"].mean() #return the mean of the "price" column in the data set grouped by age, Dwight helped with this line and the one below

#Create the column for "Total Purchase Value"
age_summ_df["Total Purchase Value"] = age_data_df.groupby(age_bins_all)["Price"].sum()

#Create the column for Avg Purchase Total per Person by Gender (Total Purchase Value / Total Players)
age_summ_df["Avg Total Purchase per Person"] = age_summ_df["Total Purchase Value"] / age_total

#Format data frame
age_summ_df["Average Purchase Price"] = age_summ_df["Average Purchase Price"].map("${:.2f}".format)
age_summ_df["Total Purchase Value"] = age_summ_df["Total Purchase Value"].map("${:.2f}".format)
age_summ_df["Avg Total Purchase per Person"] = age_summ_df["Avg Total Purchase per Person"].map("${:.2f}".format)

age_summ_df.index.name = "Age Ranges"

#Display Purchasing Analysis (Gender)
age_summ_df = age_summ_df.sort_values(by = ["Age Ranges"])
age_summ_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [18]:
#DF and grouped DF with pertinent spender information
spenders_df = data_file_df[["SN", "Price"]]
sn_spenders_df = spenders_df.groupby(["SN"])

#Calculate Values for Purchase Count, and initialize summary data frame
spender_purch_count = data_file_df["SN"].value_counts()
spender_summ_df = pd.DataFrame({"Purchase Count": spender_purch_count})

#Create column for "Average Purchase Price"
spender_summ_df["Average Purchase Price"] = sn_spenders_df.mean()

#Create column for "Total Purchase Value"
spender_summ_df["Total Purchase Value"] = sn_spenders_df.sum()

#Format data frame
spender_summ_df["Average Purchase Price"] = spender_summ_df["Average Purchase Price"].map("${:.2f}".format)

spender_summ_df = spender_summ_df.sort_values(["Total Purchase Value"], ascending = False) #Sort before format
spender_summ_df["Total Purchase Value"] = spender_summ_df["Total Purchase Value"].map("${:.2f}".format)

spender_summ_df.index.name = "SN"

#Display the spender summary DF
spender_summ_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [30]:
#Create data frame for unique item info ("Item ID", "Item Name", "Price")
item_info_df = data_file_df.drop_duplicates(subset = "Item ID")
item_info_df = item_info_df.drop(["Purchase ID", "SN", "Age", "Gender"], axis=1)
item_info_df.head(5)

,Item ID,Item Name,Price
0,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,143,Frenzied Scimitar,1.56
2,92,Final Critic,4.88
3,100,Blindscythe,3.27
4,131,Fury,1.44


In [33]:
popular_df = data_file_df[["Item ID", "Item Name", "Price"]]
items_df = popular_df.groupby("Item ID")

#top_popular = data_file_df["Item ID"].value_counts()
top_popular = popular_df["Item ID"].value_counts()

#Initialize summary data frame
popular_summ_df = pd.DataFrame({"Purchase Count": top_popular,})
popular_summ_df.index.name = "Item ID"

popular_summ_df["Total Purchase Value"] = items_df.sum()
popular_df.set_index("Item ID")

#popular_summ_df["Item Price"] = data_file_df["Price"]
popular_summ_df = pd.merge(popular_summ_df, item_info_df, on = "Item ID", how = "left")
#popular_summ_df = popular_summ_df.groupby("Item ID")


# Format dataframe
popular_summ_df["Total Purchase Value"] = popular_summ_df["Total Purchase Value"].map("${:.2f}".format)
popular_summ_df = popular_summ_df.sort_values(["Purchase Count"], ascending = False)

popular_summ_df.head(6)

,Item ID,Purchase Count,Total Purchase Value,Item Name,Price
0,92,13,$59.99,Final Critic,4.88
1,178,12,$50.76,"Oathbreaker, Last Hope of the Breaking Storm",4.23
2,108,9,$31.77,"Extraction, Quickblade Of Trembling Hands",3.53
3,82,9,$44.10,Nirvana,4.90
4,145,9,$41.22,Fiery Glass Crusader,4.58
5,132,9,$28.99,Persuasion,3.19


In [ ]:
#Identify the 5 most profitable items by total purchase value, then list (in a table):
    #Item ID
    #Item Name
    #Purchase Count
    #Item Price
    #Total Purchase Value